In [1]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import os
import pickle
import itertools
import time

import wandb
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, RandomSampler, random_split

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorWithPadding

/home/aj3051/anaconda3/envs/backpack/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BASE_MODEL = "deepseek-ai/deepseek-coder-1.3b-base"
STORAGE_DIR = "/proj/rcs-hdd/aj3051/symmetry"

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# with open(os.path.join(STORAGE_DIR, 'data_four_permutations.pickle'), 'rb') as f:
#     dataset_raw = pickle.load(f)

In [5]:
# def tokenize_dataset_individual(dataset, max_length=1024, tokenizer="deepseek-ai/deepseek-coder-1.3b-base"):
#     tokenizer = AutoTokenizer.from_pretrained(tokenizer, cache_dir="/proj/rcs-hdd/aj3051/hf_transformers")
#     res = []
#     for code, line_permutation_orders, label in tqdm(dataset):
#         tokenized_loc = [ 
#             tokenizer(line_text, return_tensors="pt", add_special_tokens=(line_num==0))['input_ids'][0]
#             for line_num, line_text in enumerate(code)
#         ]
#         input = torch.cat(tokenized_loc) # unpermuted code input
#         if len(input) > max_length:
#             continue 

#         label = tokenizer(label, return_tensors="pt", add_special_tokens=False)['input_ids'][0][0].unsqueeze(dim=0)
#         sample = (input.to(dtype=torch.long), label)
#         res.append(sample)

#     return res

# dataset = tokenize_dataset_individual(dataset=dataset_raw, max_length=1024, tokenizer="deepseek-ai/deepseek-coder-1.3b-base")

# with open(os.path.join(STORAGE_DIR, 'data_individual.pkl'), 'wb') as f:
#     pickle.dump(dataset, f)

In [4]:
with open(os.path.join(STORAGE_DIR, 'data_individual.pkl'), 'rb') as f:
    code_individual_dataset = pickle.load(f)

In [5]:
class CodeDataset(Dataset):

    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        input_ids, label = self.dataset[idx] 
        return {
            'input_ids': input_ids
            'labels': torch.tensor([label])
        }


In [6]:
dataset = CodeDataset(dataset=code_individual_dataset)

sampler = RandomSampler(dataset)
shuffled_indices = list(sampler)
shuffled_dataset = torch.utils.data.Subset(dataset, shuffled_indices)
train_length = int(0.8 * len(shuffled_dataset))
validation_length = int(0.1 * len(shuffled_dataset))
test_length = len(shuffled_dataset) - train_length - validation_length

train_set, validation_set, test_set = random_split(
    shuffled_dataset, [train_length, validation_length, test_length]
)

# train_loader = torch.utils.data.DataLoader(train_set, batch_size=None, shuffle=False)
# validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=None, shuffle=False)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=None, shuffle=False)

In [7]:
model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        cache_dir="/proj/rcs-hdd/aj3051/hf_transformers",
).to(device)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, cache_dir="/proj/rcs-hdd/aj3051/hf_transformers")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
training_args = TrainingArguments(
    output_dir='./results',         
    num_train_epochs=3,             
    per_device_train_batch_size=1,  
    per_device_eval_batch_size=1, 
    report_to=None, 
)

In [10]:
first_batch = None

In [11]:
class CustomTrainer(Trainer):
    def training_step(self, model, inputs):
        print("Input IDs shape:", inputs["input_ids"].shape)
        print("Labels shape:", inputs["labels"].shape)
        global first_batch
        first_batch = inputs["input_ids"]
        # ... rest of the code ...
        return super().training_step(model, inputs)        

In [13]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=validation_set,
    data_collator=data_collator,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aj3051. Use `wandb login --relogin` to force relogin


Input IDs shape: torch.Size([1, 145])
Labels shape: torch.Size([1, 1])


ValueError: Expected input batch_size (144) to match target batch_size (0).

In [68]:
first_batch[0]

tensor([32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014, 32014,
        32014, 32014, 32014, 32013, 13198,  7050, 15545,  8440,   821,  6337,
          334,  1232,  1753, 15545,  8440,  9828,  2039,  1232, 

In [10]:
lr = 1e-5
num_epochs = 5

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

eval_every = 3000
checkpoint_every = 2000


model.train()
step = 0
for epoch in range(num_epochs):  # Train for 5 epochs
    total_loss = 0
    # for i, batch in enumerate(tqdm(dataloader)):
    for epoch_i, batch in tqdm(enumerate(train_loader)):
        input_ids, label = batch
        input_ids = input_ids.unsqueeze(dim=0).to(device)
        label = label.to(device)
        # print(longest_len)
        optimizer.zero_grad()
        print(f"input shape: {input_ids.shape}")
        output = model(input_ids)
        logits = output.logits[:, -1, :]

        loss = loss_fn(logits, label)
        loss.backward()
        optimizer.step()

        step += 1
        # if step % eval_every == 0:
        #     val_loss = eval()
        #     wandb.log({"validation loss: ", val_loss})
        # if step % checkpoint_every == 0:
        #     save_dir = f"{MODEL_DIR}/model_{step}"
        #     torch.save({
        #         'epoch': epoch, 
        #         'model_state_dict': equitune_model.state_dict(),
        #         'optimizer_state_dict': optimizer.state_dict(),
        #         'loss': loss
            # }, save_dir)
        total_loss += loss.item()
        print(f'loss: {loss}')
        # wandb.log({"loss": loss.item()})

0it [00:00, ?it/s]

input shape: torch.Size([1, 173])


2it [00:01,  1.55it/s]

loss: 8.689229011535645
input shape: torch.Size([1, 567])
loss: nan
input shape: torch.Size([1, 205])


4it [00:01,  3.20it/s]

...

loss: nan
input shape: torch.Size([1, 615])
loss: nan
input shape: torch.Size([1, 229])


592it [01:30,  6.20it/s]

loss: nan
input shape: torch.Size([1, 230])
loss: nan
input shape: torch.Size([1, 472])


592it [01:31,  6.50it/s]


KeyboardInterrupt: 

In [ ]:
del model 